# A Guided Tour of Ray Core: Remote Objects

[*Remote Objects*](https://docs.ray.io/en/latest/walkthrough.html#objects-in-ray)
implement a [*shared-memory object store*](https://en.wikipedia.org/wiki/Shared_memory) pattern.

Objects are immutable, and can be accessed from anywhere on the cluster.

---

First, let's start Ray…

In [1]:
from icecream import ic
import logging
import ray

ray.init(
    ignore_reinit_error=True,
    logging_level=logging.ERROR,
)

{'node_ip_address': '192.168.1.248',
 'raylet_ip_address': '192.168.1.248',
 'redis_address': '192.168.1.248:6379',
 'object_store_address': '/tmp/ray/session_2021-02-23_14-49-54_812953_126759/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2021-02-23_14-49-54_812953_126759/sockets/raylet',
 'webui_url': '127.0.0.1:8265',
 'session_dir': '/tmp/ray/session_2021-02-23_14-49-54_812953_126759',
 'metrics_export_port': 43197,
 'node_id': 'e4d6b0cf552da2a71a05c21572d19a46dde4008f645d530662a05163'}

## Remote Objects example

Set up for this example...

In [10]:
import time

To start, we'll put an object into the Ray object store...

In [11]:
y = 1
obj_ref = ray.put(y)

Then get the value of this object reference

In [12]:
ray.get(obj_ref)

1

You can also access the values of multiple object references in parallel:

In [13]:
ray.get([ray.put(i) for i in range(3)])

[0, 1, 2]

Now let's set a timeout to return early from a `get` that is blocking for too long...

In [14]:
from ray.exceptions import GetTimeoutError

@ray.remote
def long_running_function ():
    time.sleep(4)

In [16]:
obj_ref = long_running_function.remote()

try:
    ray.get(obj_ref, timeout=4)
except GetTimeoutError:
    print("`get` timed out")

`get` timed out
